In [8]:
import pandas as pd
import numpy as np
import yaml
from pathlib import Path

pd.set_option("display.max_rows", 9999)
pd.set_option("display.max_colwidth", 40)
pd.set_option("display.max_columns", 999)
pd.set_option("display.precision", 2)

from db_psql_model import DatabaseCursor

PATH = list(Path().cwd().parent.glob("**/private.yaml"))[0]
OPTION = "-c search_path=dev"

MoM FFBL Weekly Rankings Replacement 

In [3]:
"""
MoM FFBL Weekly Rankings Replacement
"""
matchups_query = """SELECT * FROM dev.weeklyleaguematchups"""
teams_query = """SELECT team_key, name, nickname, game_id FROM dev.leagueteams WHERE game_id >= 390"""
matchups = DatabaseCursor(PATH, options=OPTION).copy_data_from_postgres(matchups_query)
teams = DatabaseCursor(PATH, options=OPTION).copy_data_from_postgres(teams_query)
matchups = matchups[
    (matchups["is_playoffs"] == 0)
    & (matchups["is_consolation"] == 0)
    & (matchups["game_id"] >= 390)
]
matchups_a = matchups.copy()
matchups_b = matchups.copy()
matchups_b_cols = list(matchups_b.columns)
rename_columns = {}
for col in matchups_b_cols:
    if "team_a" in col:
        rename_columns[col] = f"team_b{col[6:]}"
    elif "team_b" in col:
        rename_columns[col] = f"team_a{col[6:]}"
matchups_b.rename(columns=rename_columns, inplace=True)
matchups = pd.concat([matchups_a, matchups_b])
matchups.sort_values(["week_start", "team_a_team_key"], inplace=True)
matchups.reset_index(drop=True, inplace=True)
matchups.drop(
    [
        "is_matchup_recap_available",
        "is_tied",
        "matchup_recap_title",
        "matchup_recap_url",
        "status",
        "league_id",
    ],
    axis=1,
    inplace=True,
)
matchups["win_loss"] = np.where(
    matchups["winner_team_key"] == matchups["team_a_team_key"], "W", "L"
)
matchups["2pt_system"] = matchups.groupby(["week", "game_id"])["team_a_points"].rank(
    "first", ascending=False
)
matchups["2pt_system"] = np.where(matchups["2pt_system"] <= 5, 1, 0)
matchups = matchups[
    [
        "week",
        "team_a_team_key",
        "team_a_points",
        "team_a_projected_points",
        "win_loss",
        "2pt_system",
        "game_id",
    ]
]
weeks = list(matchups["week"].unique())
regular_season = matchups[matchups["week"] == 1]
for wk in weeks[1:]:
    reg_season_week = matchups[matchups["week"] == wk]
    regular_season = regular_season.merge(
        reg_season_week,
        how="outer",
        on="team_a_team_key",
        copy=True,
        suffixes=("", f"_{wk}"),
    )
reg_season_cols = list(regular_season.columns)
twopt_cols = []
team_pts = []
team_pro_pts = []
w_l_cols = []
for col in reg_season_cols:
    if "2pt_system" in col:
        twopt_cols.append(col)
    elif "team_a_points" in col:
        team_pts.append(col)
    elif "team_a_projected_points" in col:
        team_pro_pts.append(col)
    elif "win_loss" in col:
        w_l_cols.append(col)
regular_season.insert(2, "team_a_points_total", regular_season[team_pts].sum(axis=1))
regular_season.insert(
    3, "team_a_project_points_total", regular_season[team_pro_pts].sum(axis=1)
)
regular_season.insert(
    4, "wins", regular_season[w_l_cols].apply(lambda s: (s == "W").sum(), axis=1)
)
regular_season.insert(
    5, "losses", regular_season[w_l_cols].apply(lambda s: (s == "L").sum(), axis=1)
)
twopt_cols.append("wins")
regular_season.insert(2, "2pt_total", regular_season[twopt_cols].sum(axis=1))
week_1 = regular_season["week"]
regular_season.drop("week", axis=1, inplace=True)
regular_season.insert(6, "week_1", week_1)
regular_season.insert(
    2,
    "2pt_rank",
    regular_season.groupby(["game_id"])["2pt_total"]
    .rank(method="min", ascending=False)
    .astype(int),
)
regular_season.insert(
    4,
    "pts_rank",
    regular_season.groupby(["game_id"])["team_a_points_total"]
    .rank(method="min", ascending=False)
    .astype(int),
)
regular_season.insert(
    6,
    "projected_pts_rank",
    regular_season.groupby(["game_id"])["team_a_project_points_total"]
    .rank(method="min", ascending=False)
    .astype(int),
)
regular_season.insert(
    0,
    "tuple",
    regular_season[["2pt_rank", "pts_rank"]].apply(tuple, axis=1),
)
regular_season.insert(
    0,
    "overall_rank",
    regular_season.groupby(["game_id"])["tuple"]
    .rank(method="min", ascending=True)
    .astype(int),
)
regular_season = regular_season.merge(
    teams, how="outer", left_on="team_a_team_key", right_on="team_key"
)
team_name = regular_season["name"]
team_mang = regular_season["nickname"]
game_id = regular_season["game_id_y"]
regular_season.drop(
    ["team_a_team_key", "name", "nickname", "tuple", "game_id_x", "game_id_y"],
    axis=1,
    inplace=True,
)
regular_season.insert(1, "manager", team_mang)
regular_season.insert(2, "team_name", team_name)
regular_season.insert(0, "game_id", game_id)
regular_season.sort_values(
    ["overall_rank", "2pt_total", "team_a_points_total"],
    ascending=[True, False, False],
    inplace=True,
)

Successfully pulled: SELECT * FROM dev.weeklyleaguematchups
Successfully pulled: SELECT team_key, name, nickname, game_id FROM dev.leagueteams WHERE game_id >= 390


#MoM FFBL Draft Analysis/Trends Replacement

In [9]:
"""
MoM FFBL Draft Analysis/Trends Replacement
"""
draft_query = """SELECT * FROM dev.draftresults"""
teams_query = """SELECT * FROM dev.leagueteams"""
draft = DatabaseCursor(PATH, options=OPTION).copy_data_from_postgres(draft_query)
teams = DatabaseCursor(PATH, options=OPTION).copy_data_from_postgres(teams_query)
teams

Successfully pulled: SELECT * FROM dev.draftresults
Successfully pulled: SELECT * FROM dev.leagueteams


,clinched_playoffs,draft_grade,faab_balance,has_draft_grade,name,number_of_moves,number_of_trades,team_id,team_key,waiver_priority,roster_adds.coverage_value,roster_adds.value,team_standings.outcome_totals.losses,team_standings.outcome_totals.percentage,team_standings.outcome_totals.ties,team_standings.outcome_totals.wins,team_standings.playoff_seed,team_standings.points_against,team_standings.points_for,team_standings.rank,team_standings.streak.type,team_standings.streak.value,manager_id,game_id,league_id,nickname
0,1.0,C,34,1,Harris & Harris RBs at Law 👨🏾‍⚖️,53,0,7,406.l.103661.t.7,7,18,0,6,0.60,0,9,3.0,1560.30,1799.50,1,loss,1,7,406,103661,Greg
1,1.0,B,0,1,Fantasy Sidelines,73,2,1,406.l.103661.t.1,8,18,0,4,0.73,0,11,1.0,1658.44,1725.32,2,win,1,1,406,103661,Tim
2,1.0,B-,0,1,Dak to the Future,30,2,2,406.l.103661.t.2,1,18,0,7,0.53,0,8,4.0,1630.12,1648.42,3,loss,2,2,406,103661,Wes
3,1.0,A-,0,1,King of the Hill,54,0,4,406.l.103661.t.4,2,18,0,5,0.67,0,10,2.0,1600.38,1854.08,4,win,4,4,406,103661,Ryan
4,NaN,C,28,1,Push Ups in Prison,25,0,9,406.l.103661.t.9,3,18,0,10,0.33,0,5,8.0,1698.32,1603.22,5,loss,1,9,406,103661,Pat
5,NaN,B,0,1,Fightin’ Blue Cock/Chubb/Wood,35,0,3,406.l.103661.t.3,10,18,0,7,0.53,0,8,6.0,1662.56,1612.40,6,loss,4,3,406,103661,Kevin
6,NaN,C-,0,1,Wide Out University,37,0,6,406.l.103661.t.6,6,18,0,9,0.40,0,6,5.0,1785.84,1758.14,7,win,2,6,406,103661,Chris
7,NaN,C+,56,1,Poor Decisions 😢,32,1,8,406.l.103661.t.8,9,18,0,8,0.47,0,7,7.0,1600.58,1638.68,8,win,4,8,406,103661,Pete
8,NaN,A,44,1,Team 10,43,1,10,406.l.103661.t.10,5,18,0,9,0.40,0,6,NaN,1719.82,1500.52,8,win,1,10,406,103661,James
9,NaN,A-,60,1,The Titanic,12,0,5,406.l.103661.t.5,4,18,0,10,0.33,0,5,NaN,1685.96,1462.04,10,loss,3,5,406,103661,Jeremy


MoM FFBL League History Replacement

In [ ]:
"""
MoM FFBL League History Replacement
"""

MoM FFBL League Records Replacement

In [ ]:
"""
MoM FFBL League Records Replacement
"""